Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [14]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [15]:
INSTANCES = [
    {"cities": pd.read_csv('cities/italy.csv',   header=None, names=['name', 'lat', 'lon']), "dist_matrix": None},
    {"cities": pd.read_csv('cities/china.csv',   header=None, names=['name', 'lat', 'lon']), "dist_matrix": None},
    {"cities": pd.read_csv('cities/russia.csv',  header=None, names=['name', 'lat', 'lon']), "dist_matrix": None},
    {"cities": pd.read_csv('cities/us.csv',      header=None, names=['name', 'lat', 'lon']), "dist_matrix": None},
    {"cities": pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon']), "dist_matrix": None}
]
for instance in INSTANCES:
    cities = instance["cities"]
    dist_matrix = np.zeros((len(cities), len(cities)))
    for c1, c2 in combinations(cities.itertuples(), 2):
        dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km
    instance["dist_matrix"] = dist_matrix
    cities.head()

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [16]:
def tsp_cost(instance, tsp):
    cities = instance["cities"]
    dist_matrix = instance["dist_matrix"]
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(cities)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += dist_matrix[c1, c2]
    return tot_cost

## First Greedy Algorithm

In [17]:
def greedy_1(instance):
    cities = instance["cities"]
    dist_matrix = instance["dist_matrix"]
    visited = np.full(len(cities), False)
    dist = dist_matrix.copy()
    city = 0
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        #logging.debug(
        #    f"step: {cities.at[city,'name']} -> {cities.at[closest,'name']} ({dist_matrix[city,closest]:.2f}km)"
        #)
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    #logging.debug(
    #    f"step: {cities.at[tsp[-1],'name']} -> {cities.at[tsp[0],'name']} ({dist_matrix[tsp[-1],tsp[0]]:.2f}km)"
    #)
    tsp.append(tsp[0])


    logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(instance, tsp):.2f}km")

In [18]:
i=1
for instance in INSTANCES:
    logging.info(f"instance {i}:")
    greedy_1(instance)
    i += 1 

INFO:root:instance 1:
INFO:root:result: Found a path of 46 steps, total length 4436.03km
INFO:root:instance 2:
INFO:root:result: Found a path of 726 steps, total length 63962.92km
INFO:root:instance 3:
INFO:root:result: Found a path of 167 steps, total length 42334.16km
INFO:root:instance 4:
INFO:root:result: Found a path of 326 steps, total length 48050.03km
INFO:root:instance 5:
INFO:root:result: Found a path of 8 steps, total length 1475.53km


# EA

In [19]:
def ea(instance, population_size=100, generations=500, mutation_rate=0.1):
    cities = instance["cities"]
    dist_matrix = instance["dist_matrix"]
    num_cities = len(cities)

    # Funzione per calcolare il costo di un percorso
    def path_cost(path):
        return tsp_cost(instance, path + [path[0]])

    # Genera la popolazione iniziale con percorsi casuali
    population = [random.sample(range(num_cities), num_cities) for _ in range(population_size)]

    for generation in range(generations):
        # Calcola il costo di ogni percorso nella popolazione
        fitness_scores = [(path, path_cost(path)) for path in population]
        fitness_scores.sort(key=lambda x: x[1])  # Ordina per costo crescente
        
        # Logging del miglior percorso della generazione attuale
        #logging.debug(f"Generation {generation + 1}: Best cost = {fitness_scores[0][1]:.2f} km")

        # Seleziona metà della popolazione con i percorsi a costo più basso
        selected = [path for path, _ in fitness_scores[:population_size // 2]]

        # Crossover: combina percorsi per generare nuovi percorsi
        offspring = []
        while len(offspring) < population_size:
            parent1, parent2 = random.sample(selected, 2)
            cut = random.randint(1, num_cities - 2)
            child = parent1[:cut] + [city for city in parent2 if city not in parent1[:cut]]
            offspring.append(child)

        # Mutazione: introduce piccole modifiche nei percorsi
        for child in offspring:
            if random.random() < mutation_rate:
                i, j = random.sample(range(num_cities), 2)
                child[i], child[j] = child[j], child[i]

        # Aggiorna la popolazione
        population = offspring

    # Calcola il miglior percorso trovato
    best_path, best_cost = min(fitness_scores, key=lambda x: x[1])
    best_path.append(best_path[0])

    # Logging finale con il risultato del percorso ottimale trovato
    logging.info(f"result: Found a path of {len(best_path) - 1} steps, total length {best_cost:.2f}km")


In [20]:
i=1
for instance in INSTANCES:
    logging.info(f"instance {i}:")
    ea(instance)
    i += 1 

INFO:root:instance 1:
INFO:root:result: Found a path of 46 steps, total length 5162.24km
INFO:root:instance 2:
INFO:root:result: Found a path of 726 steps, total length 584576.85km
INFO:root:instance 3:
INFO:root:result: Found a path of 167 steps, total length 132025.59km
INFO:root:instance 4:
INFO:root:result: Found a path of 326 steps, total length 270439.71km
INFO:root:instance 5:
INFO:root:result: Found a path of 8 steps, total length 1345.54km


## Second Greedy Algorithm

In [21]:
def cyclic(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    try:
        nx.find_cycle(G)
        return True
    except:
        return False

def greedy_2(instance):
    cities = instance["cities"]
    dist_matrix = instance["dist_matrix"]
    segments = [({c1, c2}, float(dist_matrix[c1, c2])) for c1, c2 in combinations(range(len(cities)), 2)]
    visited = set()
    edges = set()
    tsp = []

    while len(visited) < len(cities):
        # Trova il segmento più corto che non forma un ciclo e aggiungilo
        shortest = next(s for s in sorted(segments, key=lambda e: e[1]) if not cyclic(edges | {tuple(s[0])}))
        edges.add(tuple(shortest[0]))
        visited |= shortest[0]

        # Aggiungi le città visitate alla sequenza tsp
        for city in shortest[0]:
            if city not in tsp:
                tsp.append(city)

        # Log ogni passo
        #logging.debug(f"step: Added edge {shortest[0]} with cost {shortest[1]:.2f}km")

        # Rimuove i segmenti che formano cicli
        segments = [s for s in segments if not cyclic(edges | {tuple(s[0])})]

    # Chiudi il ciclo tornando al punto di partenza
    tsp.append(tsp[0])
    logging.info(f"result: Found a path of {len(tsp) - 1} steps, total length {tsp_cost(instance, tsp):.2f}km")


In [22]:
#i=1
#for instance in INSTANCES:
#    logging.info(f"instance {i}:")
#    greedy_2(instance)
#    i += 1 